In [1]:
#WRANGLING-PRUEBA
import sqlite3
import pandas as pd
import numpy as np
import re
import datetime
import os
import requests
from bs4 import BeautifulSoup

In [2]:
#PASO1: ALIMENTACION - QUITAR EN .PY   /// OK 
def read_sql_table(path,table):
    df = sqlite3.connect(path)
    df_table = pd.read_sql_query('SELECT * FROM '+table,df)
    return df_table

def read_and_merge(path):
    df_business_info = read_sql_table(path, 'business_info')
    df_personal_info = read_sql_table(path, 'personal_info')
    df_rank_info = read_sql_table(path, 'rank_info')
    df4 = pd.merge(df_business_info, df_personal_info, on=['id', 'id'])
    df_dirt = pd.merge(df4, df_rank_info, on=['id', 'id'])
    df_dirt.to_csv('/Users/MIGUEL/Desktop/CLAB1/Ironhack-Module-1-Project---Pipelines-/data/df_dirt.csv', index=False)
    return df_dirt

In [3]:
#LLAMAR FUNCION ALIMENTACION,PASO1
read_and_merge('/Users/MIGUEL/Desktop/CLAB1/Ironhack-Module-1-Project---Pipelines-/data/raw/Miguel318.db')

,id,Unnamed: 0_x,Source,worth,worthChange,realTimeWorth,realTimePosition,lastName,age,Unnamed: 0_y,gender,country,image,name,position,Unnamed: 0
0,4656,52,Technology ==> Amazon,112.0 BUSD,0.0 millions USD,None,1,bEZOS,54 years old,52,M,None,https://specials-images.forbesimg.com/imageser...,jEFF BEZOS,1.0,52
1,1874,53,Technology ==> Microsoft,90.0 BUSD,-0.001 millions USD,None,2,gATES,62 years old,53,None,United States,https://specials-images.forbesimg.com/imageser...,bill GAtes,2.0,53
2,6703,54,Finance and Investments ==> Berkshire Hathaway,84.0 BUSD,-0.002 millions USD,None,3,bufFETT,87 years old,54,M,United States,https://specials-images.forbesimg.com/imageser...,WARREN BUFFETt,3.0,54
3,2637,55,Fashion & Retail ==> LVMH,72.0 BUSD,0.0 millions USD,None,4,arnAULt,69 years old,55,None,None,https://specials-images.forbesimg.com/imageser...,bernaRD ARNAULT,4.0,55
4,8242,56,Technology ==> Facebook,71.0 BUSD,0.0 millions USD,None,5,zUCKERBERG,1985,56,M,None,https://specials-images.forbesimg.com/imageser...,maRK ZUCKERBERG,5.0,56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2203,1711,2255,"Fashion & Retail ==> fashion, entertainment",1.0 BUSD,0.0 millions USD,None,2134,ZHAO,51 years old,2255,None,China,https://specials-images.forbesimg.com/imageser...,zhao xiAOQIang,2204.0,2255
2204,3538,2256,Manufacturing ==> electrical equipment,1.0 BUSD,nan millions USD,None,2134,zhOU,55 years old,2256,M,None,https://specials-images.forbesimg.com/imageser...,zhou LIANGZHang,2205.0,2256
2205,7797,2257,Manufacturing ==> electrical equipment,1.0 BUSD,0.0 millions USD,None,1856,ZHU,51 years old,2257,None,China,https://specials-images.forbesimg.com/imageser...,zHU XINGMing,2206.0,2257
2206,3123,2258,Manufacturing ==> printed circuit boards,1.0 BUSD,0.0 millions USD,None,1978,ZHUO,52 years old,2258,F,None,https://specials-images.forbesimg.com/imageser...,ZHUO JUN,2207.0,2258


In [4]:
#PASO INTERMEDIO - NO .PY - EXTRAER df_dirt
df_dirt = pd.read_csv('/Users/MIGUEL/Desktop/CLAB1/Ironhack-Module-1-Project---Pipelines-/data/df_dirt.csv')

In [5]:
#PASO2: drop columns /// OK
def drop_and_clean(df, list_of_columns):   
    df.drop(list_of_columns, axis=1)
    return df

In [6]:
#LLAMAR FUNCION PASO2
columns_to_drop=['id','realTimeWorth','Unnamed: 0','Unnamed: 0_x','Unnamed: 0_y','realTimePosition','lastName']
df2 = drop_and_clean(df_dirt,columns_to_drop)
df2

,id,Unnamed: 0_x,Source,worth,worthChange,realTimeWorth,realTimePosition,lastName,age,Unnamed: 0_y,gender,country,image,name,position,Unnamed: 0
0,4656,52,Technology ==> Amazon,112.0 BUSD,0.0 millions USD,NaN,1,bEZOS,54 years old,52,M,None,https://specials-images.forbesimg.com/imageser...,jEFF BEZOS,1.0,52
1,1874,53,Technology ==> Microsoft,90.0 BUSD,-0.001 millions USD,NaN,2,gATES,62 years old,53,None,United States,https://specials-images.forbesimg.com/imageser...,bill GAtes,2.0,53
2,6703,54,Finance and Investments ==> Berkshire Hathaway,84.0 BUSD,-0.002 millions USD,NaN,3,bufFETT,87 years old,54,M,United States,https://specials-images.forbesimg.com/imageser...,WARREN BUFFETt,3.0,54
3,2637,55,Fashion & Retail ==> LVMH,72.0 BUSD,0.0 millions USD,NaN,4,arnAULt,69 years old,55,None,None,https://specials-images.forbesimg.com/imageser...,bernaRD ARNAULT,4.0,55
4,8242,56,Technology ==> Facebook,71.0 BUSD,0.0 millions USD,NaN,5,zUCKERBERG,1985,56,M,None,https://specials-images.forbesimg.com/imageser...,maRK ZUCKERBERG,5.0,56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2203,1711,2255,"Fashion & Retail ==> fashion, entertainment",1.0 BUSD,0.0 millions USD,NaN,2134,ZHAO,51 years old,2255,None,China,https://specials-images.forbesimg.com/imageser...,zhao xiAOQIang,2204.0,2255
2204,3538,2256,Manufacturing ==> electrical equipment,1.0 BUSD,nan millions USD,NaN,2134,zhOU,55 years old,2256,M,None,https://specials-images.forbesimg.com/imageser...,zhou LIANGZHang,2205.0,2256
2205,7797,2257,Manufacturing ==> electrical equipment,1.0 BUSD,0.0 millions USD,NaN,1856,ZHU,51 years old,2257,None,China,https://specials-images.forbesimg.com/imageser...,zHU XINGMing,2206.0,2257
2206,3123,2258,Manufacturing ==> printed circuit boards,1.0 BUSD,0.0 millions USD,NaN,1978,ZHUO,52 years old,2258,F,None,https://specials-images.forbesimg.com/imageser...,ZHUO JUN,2207.0,2258


In [7]:
#PASO3: WRANGLING - wrangling: col1,worth, col2,worthchange: /// OK

def repla_cols(df, col1, col2):
    df[col1].replace({"BUSD": ' '})
    df[col2].replace('na', '0.0', regex=True, inplace=True)
    df[col1].replace('[BUSD]+$', '', regex=True, inplace=True)
    df[col2].replace('[millions USD]+$', '', regex=True, inplace=True)
    df[col2].astype('float').astype('int')
    df[col2].astype('float').astype('int')
    return df

In [8]:
#LLAMAR FUNCION-replacement, PASO2
repla_cols(df2, 'worth', 'worthChange')

,id,Unnamed: 0_x,Source,worth,worthChange,realTimeWorth,realTimePosition,lastName,age,Unnamed: 0_y,gender,country,image,name,position,Unnamed: 0
0,4656,52,Technology ==> Amazon,112.0,0.0,NaN,1,bEZOS,54 years old,52,M,None,https://specials-images.forbesimg.com/imageser...,jEFF BEZOS,1.0,52
1,1874,53,Technology ==> Microsoft,90.0,-0.001,NaN,2,gATES,62 years old,53,None,United States,https://specials-images.forbesimg.com/imageser...,bill GAtes,2.0,53
2,6703,54,Finance and Investments ==> Berkshire Hathaway,84.0,-0.002,NaN,3,bufFETT,87 years old,54,M,United States,https://specials-images.forbesimg.com/imageser...,WARREN BUFFETt,3.0,54
3,2637,55,Fashion & Retail ==> LVMH,72.0,0.0,NaN,4,arnAULt,69 years old,55,None,None,https://specials-images.forbesimg.com/imageser...,bernaRD ARNAULT,4.0,55
4,8242,56,Technology ==> Facebook,71.0,0.0,NaN,5,zUCKERBERG,1985,56,M,None,https://specials-images.forbesimg.com/imageser...,maRK ZUCKERBERG,5.0,56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2203,1711,2255,"Fashion & Retail ==> fashion, entertainment",1.0,0.0,NaN,2134,ZHAO,51 years old,2255,None,China,https://specials-images.forbesimg.com/imageser...,zhao xiAOQIang,2204.0,2255
2204,3538,2256,Manufacturing ==> electrical equipment,1.0,0.0,NaN,2134,zhOU,55 years old,2256,M,None,https://specials-images.forbesimg.com/imageser...,zhou LIANGZHang,2205.0,2256
2205,7797,2257,Manufacturing ==> electrical equipment,1.0,0.0,NaN,1856,ZHU,51 years old,2257,None,China,https://specials-images.forbesimg.com/imageser...,zHU XINGMing,2206.0,2257
2206,3123,2258,Manufacturing ==> printed circuit boards,1.0,0.0,NaN,1978,ZHUO,52 years old,2258,F,None,https://specials-images.forbesimg.com/imageser...,ZHUO JUN,2207.0,2258


In [9]:
#PASO4: FUNCION RENAME COLS - worth-worthchange
def rename2(df):
    df.rename(columns={'worth':'worth BUSD', 'worthChange':' worthChange millions USD'})
    return df

In [10]:
#LLAMAR FUNCION
rename2(df2)

,id,Unnamed: 0_x,Source,worth,worthChange,realTimeWorth,realTimePosition,lastName,age,Unnamed: 0_y,gender,country,image,name,position,Unnamed: 0
0,4656,52,Technology ==> Amazon,112.0,0.0,NaN,1,bEZOS,54 years old,52,M,None,https://specials-images.forbesimg.com/imageser...,jEFF BEZOS,1.0,52
1,1874,53,Technology ==> Microsoft,90.0,-0.001,NaN,2,gATES,62 years old,53,None,United States,https://specials-images.forbesimg.com/imageser...,bill GAtes,2.0,53
2,6703,54,Finance and Investments ==> Berkshire Hathaway,84.0,-0.002,NaN,3,bufFETT,87 years old,54,M,United States,https://specials-images.forbesimg.com/imageser...,WARREN BUFFETt,3.0,54
3,2637,55,Fashion & Retail ==> LVMH,72.0,0.0,NaN,4,arnAULt,69 years old,55,None,None,https://specials-images.forbesimg.com/imageser...,bernaRD ARNAULT,4.0,55
4,8242,56,Technology ==> Facebook,71.0,0.0,NaN,5,zUCKERBERG,1985,56,M,None,https://specials-images.forbesimg.com/imageser...,maRK ZUCKERBERG,5.0,56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2203,1711,2255,"Fashion & Retail ==> fashion, entertainment",1.0,0.0,NaN,2134,ZHAO,51 years old,2255,None,China,https://specials-images.forbesimg.com/imageser...,zhao xiAOQIang,2204.0,2255
2204,3538,2256,Manufacturing ==> electrical equipment,1.0,0.0,NaN,2134,zhOU,55 years old,2256,M,None,https://specials-images.forbesimg.com/imageser...,zhou LIANGZHang,2205.0,2256
2205,7797,2257,Manufacturing ==> electrical equipment,1.0,0.0,NaN,1856,ZHU,51 years old,2257,None,China,https://specials-images.forbesimg.com/imageser...,zHU XINGMing,2206.0,2257
2206,3123,2258,Manufacturing ==> printed circuit boards,1.0,0.0,NaN,1978,ZHUO,52 years old,2258,F,None,https://specials-images.forbesimg.com/imageser...,ZHUO JUN,2207.0,2258


In [11]:
#PASO3: FUNCION REPLACEMENT: wrangling - gender:  /// OK
def wrangling_gender(df, column):
    df[column].replace('[M]+$', 'Male', regex=True, inplace=True)
    df[column].replace('[F]+$', 'Female', regex=True, inplace=True)
    df[column].astype('object').astype('str')
    return df

In [12]:
#PASO: LLAMAR FUNCIÓN REPLACEMENT
wrangling_gender(df2, 'gender')

,id,Unnamed: 0_x,Source,worth,worthChange,realTimeWorth,realTimePosition,lastName,age,Unnamed: 0_y,gender,country,image,name,position,Unnamed: 0
0,4656,52,Technology ==> Amazon,112.0,0.0,NaN,1,bEZOS,54 years old,52,Male,None,https://specials-images.forbesimg.com/imageser...,jEFF BEZOS,1.0,52
1,1874,53,Technology ==> Microsoft,90.0,-0.001,NaN,2,gATES,62 years old,53,None,United States,https://specials-images.forbesimg.com/imageser...,bill GAtes,2.0,53
2,6703,54,Finance and Investments ==> Berkshire Hathaway,84.0,-0.002,NaN,3,bufFETT,87 years old,54,Male,United States,https://specials-images.forbesimg.com/imageser...,WARREN BUFFETt,3.0,54
3,2637,55,Fashion & Retail ==> LVMH,72.0,0.0,NaN,4,arnAULt,69 years old,55,None,None,https://specials-images.forbesimg.com/imageser...,bernaRD ARNAULT,4.0,55
4,8242,56,Technology ==> Facebook,71.0,0.0,NaN,5,zUCKERBERG,1985,56,Male,None,https://specials-images.forbesimg.com/imageser...,maRK ZUCKERBERG,5.0,56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2203,1711,2255,"Fashion & Retail ==> fashion, entertainment",1.0,0.0,NaN,2134,ZHAO,51 years old,2255,None,China,https://specials-images.forbesimg.com/imageser...,zhao xiAOQIang,2204.0,2255
2204,3538,2256,Manufacturing ==> electrical equipment,1.0,0.0,NaN,2134,zhOU,55 years old,2256,Male,None,https://specials-images.forbesimg.com/imageser...,zhou LIANGZHang,2205.0,2256
2205,7797,2257,Manufacturing ==> electrical equipment,1.0,0.0,NaN,1856,ZHU,51 years old,2257,None,China,https://specials-images.forbesimg.com/imageser...,zHU XINGMing,2206.0,2257
2206,3123,2258,Manufacturing ==> printed circuit boards,1.0,0.0,NaN,1978,ZHUO,52 years old,2258,Female,None,https://specials-images.forbesimg.com/imageser...,ZHUO JUN,2207.0,2258


In [13]:
#PASO4: FUNCION - wrangling - country:
def non_values(df, column):
    df[column].replace('None', np.nan, regex=True, inplace=True)
    df_no_null_values = df[df[column].isnull()] 
    df.drop(df[df[column].isnull()].index, inplace = True)
    return df

In [14]:
#LLAMAR FUNCION - non_values
non_values(df2, 'country')

,id,Unnamed: 0_x,Source,worth,worthChange,realTimeWorth,realTimePosition,lastName,age,Unnamed: 0_y,gender,country,image,name,position,Unnamed: 0
1,1874,53,Technology ==> Microsoft,90.0,-0.001,NaN,2,gATES,62 years old,53,None,United States,https://specials-images.forbesimg.com/imageser...,bill GAtes,2.0,53
2,6703,54,Finance and Investments ==> Berkshire Hathaway,84.0,-0.002,NaN,3,bufFETT,87 years old,54,Male,United States,https://specials-images.forbesimg.com/imageser...,WARREN BUFFETt,3.0,54
6,3663,58,Telecom ==> telecom,67.1,0.001,NaN,7,slim HELu,78 years old,58,Male,Mexico,https://specials-images.forbesimg.com/imageser...,carlos sLIM HELU,7.0,58
7,5909,59,Diversified ==> Koch Industries,60.0,0.0,NaN,12,koCH,82 years old,59,Male,USA,https://specials-images.forbesimg.com/imageser...,chaRLES koch,8.0,59
18,9144,70,"Energy ==> petrochemicals, oil & gas",40.1,0.0,NaN,17,amBANI,61 years old,70,Male,India,https://specials-images.forbesimg.com/imageser...,mukesH AMBANI,19.0,70
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2191,8246,2243,Real Estate ==> real estate,1.0,0.0,NaN,2138,taraSCio,73 years old,2243,Male,Australia,https://specials-images.forbesimg.com/imageser...,sam taRASCio,2192.0,2243
2194,8970,2246,Finance and Investments ==> investments,1.0,0.0,NaN,2146,vIK,63 years old,2246,Male,Norway,https://specials-images.forbesimg.com/imageser...,ALEXANDER VIK,2195.0,2246
2202,1174,2254,Diversified ==> air conditioning,1.0,0.0,NaN,2114,yAO,53 years old,2254,Male,People's Republic of China,https://specials-images.forbesimg.com/imageser...,yAO XINYi,2203.0,2254
2203,1711,2255,"Fashion & Retail ==> fashion, entertainment",1.0,0.0,NaN,2134,ZHAO,51 years old,2255,None,China,https://specials-images.forbesimg.com/imageser...,zhao xiAOQIang,2204.0,2255
